TODO:
- Describe HBc.. Columns

In [30]:
# imports, libraries
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
import regex as re
from sklearn.preprocessing import StandardScaler, Binarizer, LabelEncoder, Normalizer, OneHotEncoder
from sklearn.impute import SimpleImputer
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
import os

# import helperfunctions
import preprocessing.pipe as pipe

In [31]:
os.getcwd()
os.chdir("D:/Drive/FHNW/zaRepos/fhnw_ds_fs2021_medical_challenge/preprocessing")

## Import and Renaming
The dataset is imported with pandas `read_excel()`. The naming of the features, i.e. the names of the columns is not uniform. The features are renamed with the function `pipe.rename()`, which can be found in the script pipe.py, based on a given list. The list can be consulted in the document "col_names&data_type-Copy1.xlsx". All features are renamed in lowercase, and preceding and trailing spaces are removed. Brackets and their contents, e.g. "(Blood)", are removed. These would only complicate the readability of the code and are recognizable from the context as well as the name of the feature.

In [32]:
# import dataframe
df = pd.read_excel("../data/uveitis_data.xlsx")
assert len(df) >= 1075, "Data is not complete"

# rename columns
df = pipe.rename(df, "../data/col_names&data_type-Copy1.xlsx")

In [33]:
df = df.applymap(lambda x: x.strip() if isinstance(x, str) else x)

In [34]:
df = pipe.drop_nan_columns(df, nan_percentage=.5, verbose = True)

The following columns have been removed from the dataset:

                       index     ratio
70              anti-dnase_b  0.996279
5                     other_  0.790698
8                      notes  0.790698
60      beta-2-microglobulin  0.657674
63       lupus_anticoagulant  0.651163
106       myeloperoxidase_ab  0.624186
109  proteinase-3_antibodies  0.621395


In [35]:
# df = pipe.drop_via_filter(test, 'range', verbose=True)

# Categorical Features
This section deals with categorical variables that can be taken as such directly from the dataset. There are features/variables that contain both categorical and numerical values. These are treated seperately. For each feature, a description is given of how it was processed. Mostly it is a simple normalization of the values, uniformization of values that contain the same information or removal of wrong or useless values. The decision to evaluate a value as "missing" is discussed in each case. All changes made can be adjusted or undone.



## Feature Description 
- **Gender**, a qualitative, nominal feature describing the patients gender. A patient can either be in the "male" or "female" category.

- **Race** describes the patients ethnicity.

- **Location** locates the position of the inflammation in the eye. A distinction is made between posterior, anterior, intermediate, etc. 

- The feature **Categorical** records the source of the inflammation as seen by the specialists who recorded the data. Uveitis can be caused by systemic problems, infections, or often idopathic.

- **EHR Diagnosos** is an electronic transmited diagnose, usually given beforehand by another doctor, that has had no knowledge about the lab tests and final diagnosis.

- **Specific Diagnosis** is the diagnosis given by the team that collected the data. According to Dr. Nida Sen this is one of the most important outcome variables. This variable will be consired to be the target feature. 

- **AC Abn Od Cells and AC Abn Os Cells**. These qualitative, ordinal features describe the severity of the inflammation of the Anterior Chamber Cells (AC) in either the left eye (OS) or the right eye (OD). The inflammation can be rated as 0, +0.5, +1, +2, +3, +4. The higher the value the more severe the inflammation is. If either one of these values a patient can be considered as "Active", else as "Quiet". This information could be recorded in a new column.

- **Vit Abn Od Cells, Vit Abn Os Cells, Vit Abn Od Haze and Vit Abn Os Haze** describe (similar to AC Abn O...) the inflammation of cells in the left (OS) and right (OD) eye. The same scale of 0, +0.5, +1, +2, +3, +4 is used. If one of the values is higher than 0 the patient is considered to be "Active" as well. This information can be recorded in a new column as well. 

- **HBc (HepB core) Ab (Blood), HBs (HepB surface) Ag (Blood), HCV (HepC) Ab (Blood)** 

Features that contain categorical and numerical information will be discussed in a later chapter.

### Gender

This features containes the gender of the patient ("female" or "male") and is currently of the data type 'Object' ('O'). This feature gets transfromed to the dtype 'catgory' via the `pd.DataFrame.astype('category')`-function. This way it can later on easily be OneHotEncoded. 

In [36]:
df.gender.unique().tolist() # categories in feature 'gender'

['Male', 'Female']

In [37]:
df.gender.dtype # dtype before transformation

dtype('O')

In [38]:
# gender
def gender_dtype(df):
    df.gender = df.gender.astype('category')
    return df

df = gender_dtype(df)
df.gender.dtype # dtype after transformation

CategoricalDtype(categories=['Female', 'Male'], ordered=False)

### Race
The categorical variable "Race" includes the category "race or ethnic group data not provided by source". These values are treated as missing values, aka in the category 'unknown', since they do not contain any information about the respective person. "race or ethnic group data not provided by source" and "unknown race" collaps into the category "unknown". Missing values (NaN's) are also marked with 'unknown'

In [39]:
def race_processing(df):
    df.race = df.race.replace({'Race or Ethnic Group Data Not Provided by Source':'unknown', 
                               'Unknown Race':'unknown'})
    df.race = df.race.fillna(value='unknown')
    df.race = df.race.astype('category')
    assert df.race.isna().sum() == 0, 'Not all missing values are treated'
    return df
    
df = race_processing(df)
df.race.value_counts()

White                                        490
Black/African American                       381
Asian                                         94
unknown                                       75
Multiracial                                   31
Native Hawaiian or Other Pacific Islander      3
American Indian or Alaska Native               1
Name: race, dtype: int64

TODO: Categories with less than 10 values, aka 'Native Hawaiian or Other Pacific Islander', 'American Indian or Alaska Native' may should be collapsed or discarded,

### loc, "Location"
The loc-Feature indicates the location of the inflammation of the eye. The category 'pan' is the same as 'panuveitis' and can be collapsed. 
We want to explore two diffrent approaches to treat this feature:

1. We keep the categories 'anterior', 'intermediate', 'panuveitis', 'posterior' and 'sclerits'. All categories indicate a diffrent section of the eye (or multiple at once) that show inflammation. 
2. We collapse mutliple categories to get an 'anterior' and 'posterior' category. Aka, collapse the location to inflammations in the front and the back of the eye (binary feature). To achieve this we collapse the categories 'intermediate', 'posterior' and 'panuveities' to the category "posterior_segment". 'anterior' and 'scleritis' get collapsed to the category 'anterior_segment'.

In [40]:
df = pipe.preprocessing_loc(df,'multi', verbose=True)

Categories: 

posterior       341
anterior        242
panuveitis      228
intermediate    182
scleritis        71
Name: loc, dtype: int64


### cat, "Category" 
The cat-feature describes the origin of the inflammation. For example infectious or idiopathic origin. 
We can collapse the categories "nonneoplastic masquerade" and " neoplastic masquerade" to not_uveitits. As these are "pseudo-uveitis"-types. The row with the single occurance of scleritis should be dropped as it has to few records with this category. The single occurance of NaN is a "not_uveitis" case and can be filled with that category.

In [41]:
for i in range(2):
    df.cat = df.cat.str.lower().str.strip().astype('category')
    if df.cat.isna().sum() >= 1:
        df.cat = df.cat.fillna(value='not_uveitis')
    df.loc[df['cat'].str.contains('masquerade', case=False), 'cat'] = 'not_uveitis'
    df.drop(df[df.cat == 'scleritis'].index, inplace = True)
    df.cat.value_counts()
df.cat.value_counts()

idiopathic     530
systemic       192
not_uveitis    141
wds            124
infectious      87
Name: cat, dtype: int64

### ehr_diagnosis
EHR diagnosos is an electronic transmitted diagnosis, usually given beforehand by another doctor, not knowing about the lab results and final diagnosis. This feature contains a lot of diffrent categories (533 unique values). Because of that we drop this feature.

In [42]:
df.drop(columns=['ehr_diagnosis'], inplace=True)

### specific_diagnosis
Specific diagnoses which occur less or equal to 10 times in the dataset get collapsed into the catgory 'other'

In [43]:
def preprocessing_specific(df):
    df.specific_diagnosis = df.specific_diagnosis.str.lower().astype('category')
    df.loc[df['specific_diagnosis'].str.contains('masquerade', case=False), 'specific_diagnosis'] = 'not_uveitis'

    count = df.specific_diagnosis.value_counts().reset_index().rename(columns={'index':'diagnosis','specific_diagnosis':'count'})
    diag_less_10 = count[count['count'] <= 10].diagnosis.tolist()
    df.specific_diagnosis = df.specific_diagnosis.replace({x:'other' for x in diag_less_10})
    return df

df = preprocessing_specific(df)
df.specific_diagnosis.value_counts()

idiopathic_anterior      169
pars planitis            145
not_uveitis              137
idiopathic_panuveitis    107
wds                       92
idiopathic_posterior      58
idiopathic_scleritis      56
vkh                       45
toxoplasmosis             34
presumed_sarcoidosis      33
bcr                       32
other                     26
bx_proven_sarcoidosis     26
viral                     25
rheumatologic             24
hla_b27                   21
behcet                    19
tuberculosis              13
multiple_sclerosis        12
Name: specific_diagnosis, dtype: int64

### notes
This column contains notes to the diagnosis and is mostly missing. This feature will be dropped at the end of the preprocessing.

In [44]:
if 'notes' in df.columns:
    print(df.note.isna().sum()/len(df))

### ac_abn_...-columns and vit_abn_...-columns
Replace 'C' as Missing and change dtype to 'float'

In [45]:
def preprocessing_inflammation(df, col = ['ac_abn_od_cells', 'ac_abn_os_cells', 'vit_abn_od_cells',
       'vit_abn_os_cells', 'vit_abn_od_haze', 'vit_abn_os_haze']):
    for c in col: 
        # replace 'C' (for missing) with NaN
        df[c] = df[c].replace('C',np.nan)
        df[c] = df[c].astype('float')
        df[c] = pd.Categorical(values=df[c], categories=df[c].unique().sort(), ordered=True)
    return df
df = preprocessing_inflammation(df)

### hbc__ab, hbs__ag and hcv__ab
These columns encode the lab results for diffrent types of hepatitis. We encode these in binary form. Negative results are '0' and positive results get encoded as '1'. There are some cases where neither a positive or negative result can be identified. These values will be set as missing values. 

In [46]:
def preprocessing_hepatitis(df, col=['hbc__ab', 'hbs__ag', 'hcv__ab'], verbose=False):
    for c in col:
        df[c] = df[c].str.lower()
        df.loc[df[c] == 'negative', c] = 0
        df.loc[df[c] == 'see note | positive result s/co ratio is >5.0.  confirmatory testing i', c] = 1
        df.loc[df[c] == 'see below | positive result s/co ratio is >5.0.  confirmatory testing', c] = 1
        df.loc[df[c] == 'reactive', c] = 1
        df.loc[df[c] == 'repeat reactive', c] = 1
        df.loc[df[c] == 'invalid result', c] = np.nan
        df.loc[df[c] == 'note:', c] = np.nan
        df[c] = df[c].astype('category')
        if verbose:
            print(df[c].value_counts())
    return df
df = preprocessing_hepatitis(df)

### hla-columns
These columns contain genetic data about the patients. This data should be used for a seperate model and thus will not be used (at least for now) and dropped. A function has been defined to drop these columns.

In [47]:
# df = pipe.drop_via_filter(test, 'hla', verbose=True)

## Numerical Features
This section deals with numerical variables that can be extracted from the dataset. As mentioned before, there are features/variables that contain both categorical and numerical values. These are treated seperately. For each feature, a description is given of how it was processed. Mostly it is a simple normalization of the values, uniformization of values that contain the same information or removal of wrong or useless values. The decision to evaluate a value as "missing" is discussed in each case. All changes made can be adjusted or undone.

### Feauture Description


In [48]:
#variables assignement
list_path = "../data/col_names&data_type-Copy1.xlsx"
col_index_name = "new col name"
col_data_type_name = "data_type"
data_type = "numerical"

desired_dtype = ["Int64", "float64"]

#return list of all columns with specific dtype
num_columns = pipe.list_of_totype(list_path, col_index_name, col_data_type_name, data_type)
print(num_columns)

#create dataframe with columns that contain a mix of strings and numerical values
problem_df = pipe.coerce_then_problems(df, list_path, col_index_name, col_data_type_name, data_type, desired_dtype)
problem_columns = list(problem_df)
print(problem_columns)

df

['id', 'calcium', 'lactate_dehydrogenase', 'c-reactive_protein,_normal_and_high_sensitivity', 'wbc', 'rbc', 'hemoglobin', 'hematocrit', 'mcv', 'mch', 'mchc', 'rdw', 'platelet_count', 'neutrophil_%', 'lymphocytes_%', 'angiotensin_conv#enzyme', 'beta-2-microglobulin', 'lysozyme,_plasma', 'anti-dnase_b', 'complement_c3', 'complement_c4', 'rheumatoid_factor']
        id calcium lactate_dehydrogenase  \
0        1    2.27                   184   
1        2    2.32                   209   
2        3    2.44                   194   
3        4    2.45                   156   
4        5    2.28                   175   
...    ...     ...                   ...   
1070  1071    2.26                   194   
1071  1072    2.36                   239   
1072  1073    2.26                   196   
1073  1074    2.41                   173   
1074  1075    2.39                   214   

     c-reactive_protein,_normal_and_high_sensitivity    wbc   rbc hemoglobin  \
0                                

,id,gender,race,loc,cat,specific_diagnosis,ac_abn_od_cells,ac_abn_os_cells,vit_abn_od_cells,vit_abn_os_cells,vit_abn_od_haze,vit_abn_os_haze,calcium,uom,range,lactate_dehydrogenase,uom1,range1,"c-reactive_protein,_normal_and_high_sensitivity",uom2,range2,wbc,uom3,range3,rbc,uom4,range4,hemoglobin,uom5,range5,hematocrit,uom6,range6,mcv,uom7,range7,mch,uom8,range8,mchc,uom9,range9,rdw,uom10,range10,platelet_count,uom11,range11,neutrophil_%,uom12,range12,lymphocytes_%,uom13,range13,angiotensin_conv#enzyme,uom14,range14,uom15,range15,"lysozyme,_plasma",uom18,range18,anti-ccp_ab,uom21,range21,uom22,range22,anti-ena_screen,uom23,range23,antinuclear_antibody,uom24,range24,complement_c3,uom25,range25,complement_c4,uom26,range26,dna_double-stranded_ab,uom27,range27,hla-a*,hla_a_1,hla_a_2,hla-b*,hla_b_1,hla_b_2,hla-cw*,hla_c_1,hla_c_2,hla-drb1*,hla_drb1_1,hla_drb1_2,hla-dqb1*_/_dq*,hla_dq_1,hla_dq_2,hla-drb_*,hla_drb*_1,hla_drb*_2,uom34,range34,uom35,range35,rheumatoid_factor,uom36,range36,hbc__ab,hbs__ag,hcv__ab
0,1,Male,Black/African American,anterior,not_uveitis,idiopathic_anterior,0.0,0.0,0.0,0.5,0.0,0.0,2.27,mmol/L,2.05-2.50,184,U/L,113-226,1.83,mg/L,0.00-4.99,6.91,K/uL,4.23-9.07,4.97,M/uL,4.63-6.08,14.6,g/dL,13.7-17.5,41.4,%,40.1-51.0,83.3,fL,79.0-92.2,29.4,pg,25.7-32.2,35.3,g/dL,32.3-36.5,13.9,%,11.6-14.4,194,K/uL,161-347,64.4,%,34.0-67.9,30.8,%,21.8-53.1,NaN,NaN,NaN,mg/L,0.9-1.7,3,mcg/mL,2.7 - 9.4,<20,Unit,0-20,NaN,NaN,NEG,EU,0-19,NEG,EU,0.0-0.9,149.0,mg/dL,90.0-180.0,33,mg/dL,10.0-40.0,NEG,IU/mL,0-30,"30,74",30.0,74.0,"08,82:01",8.0,82.0,"03,07",3.0,7.0,"03*,13*",3.0,13.0,"02:01,05*",2.0,5.0,"3*02,3*03:01",3.0,3.0,NaN,NaN,NaN,NaN,<15,IU/mL,<15,0,0,0
1,2,Female,Black/African American,intermediate,idiopathic,pars planitis,2.0,2.0,2.0,2.0,0.5,1.0,2.32,mmol/L,2.15-2.55,209,U/L,120-290,<0.15,mg/L,0.00-4.99,4.24,K/uL,3.98-10.04,4.22,M/uL,3.93-5.22,11,g/dL,11.2-15.7,36,%,34.1-44.9,85.3,fL,79.4-94.8,26.1,pg,25.6-32.2,30.6,g/dL,32.2-35.5,14.8,%,11.7-14.4,260,K/uL,173-369,40.1,%,34.0-71.1,49.1,%,19.3-51.7,27.3,U/L,3.0-52.0,NaN,NaN,NaN,NaN,NaN,<20,Unit,0-19,NaN,NaN,NEG,EU,0-19,NEG,EU,0.0-0.9,130.8,mg/dL,90.0-180.0,22.8,mg/dL,10.0-40.0,NEG,IU/mL,0-30,"01,24",1.0,24.0,"07,41",7.0,41.0,"06,17",6.0,17.0,"01*,07*",1.0,7.0,"03:03:02,05:01*",3.0,5.0,4*01:03:01:02N,4.0,4.0,U,<0.4 (Negative),U,<0.4 (Negative),<10,IU/mL,<13,0,0,0
2,3,Female,White,panuveitis,systemic,vkh,0.5,0.5,1.0,1.0,0.0,0.0,2.44,mmol/L,2.15-2.55,194,U/L,113-226,0.137,mg/L,0.00-4.99,6.38,K/uL,3.98-10.04,4.31,M/uL,3.93-5.22,13.6,g/dL,11.2-15.7,38.7,%,34.1-44.9,89.8,fL,79.4-94.8,31.6,pg,25.6-32.2,35.1,g/dL,32.2-35.5,11.7,%,11.7-14.4,344,K/uL,173-369,70.2,%,34.0-71.1,22.1,%,19.3-51.7,60.2,U/L,3.0-52.0,NaN,NaN,4.8,mcg/mL,2.7 - 9.4,<20,Unit,0-19,NaN,NaN,NEG,EU,0-19,NEG,EU,0.0-0.9,135.9,mg/dL,90.0-180.0,18.2,mg/dL,10.0-40.0,NEG,IU/mL,0-30,"23,33",23.0,33.0,"44,51",44.0,51.0,"02,04",2.0,4.0,4*01,4.0,4.0,"03*,05*",3.0,5.0,"01:01:01,04:08:01",1.0,4.0,NaN,NaN,NaN,NaN,<10,IU/mL,<13,0,0,0
3,4,Female,White,anterior,idiopathic,idiopathic_anterior,0.0,0.0,0.0,0.0,0.0,0.0,2.45,mmol/L,2.15-2.55,156,U/L,113-226,3.5,mg/L,0.00-4.99,5.24,K/uL,3.98-10.04,4.82,M/uL,3.93-5.22,14.6,g/dL,11.2-15.7,43.6,%,34.1-44.9,90.5,fL,79.4-94.8,30.3,pg,25.6-32.2,33.5,g/dL,32.2-35.5,12.4,%,11.7-14.4,220,K/uL,173-369,56.6,%,34.0-71.1,34.2,%,19.3-51.7,38.9,U/L,3.0-52.0,NaN,NaN,:,NaN,NaN,<20,Unit,0-19,NaN,NaN,NEG,EU,0-19,NEG,EU,0.0-0.9,130.0,mg/dL,90.0-180.0,19.9,mg/dL,10.0-40.0,NEG,IU/mL,0-30,"02,32",2.0,32.0,"44,49",44.0,49.0,"05,07",5.0,7.0,"04*,14*",4.0,14.0,"03:05:01,05:03:01",3.0,5.0,"3*02,4*01",3.0,4.0,NaN,NaN,NaN,NaN,<15,IU/mL,<15,0,0,0
4,5,Female,unknown,anterior,idiopathic,idiopathic_anterior,0.0,0.0,0.0,0.0,0.0,0.0,2.28,mmol/L,2.15-2.55,175,U/L,113-226,1,mg/L,0.00-4.99,8.31,K/uL,3.98-10.04,4.72,M/uL,3.93-5.22,14,g/dL,11.2-15.7,40.7,%,34.1-44.9,86.2,fL,79.4-94.8,29.7,pg,25.6-32.2,34.4,g/dL,32.2-35.5,12.1,%,11.7-14.4,324,K/uL,173-369,60.4,%,34.0-71.1,27.4,%,19.3-51.7,18.3,U/L,3.0-52.0,NaN,NaN,4.5,mcg/mL,2.7 - 9.4

In [62]:
corrected_df = pipe.iter_columns_extract_num(problem_df)
corrected_df

#foo = pipe.coerce_then_problems(df, list_path, col_index_name, col_data_type_name, data_type, desired_dtype)

df = pipe.merge_corrected(df, corrected_df)
df

float64
Int64
float64
float64
float64
float64
float64
float64
float64
float64
float64
Int64
float64
float64
float64
float64
float64
Int64
        id  calcium  lactate_dehydrogenase  \
0        1     2.27                    184   
1        2     2.32                    209   
2        3     2.44                    194   
3        4     2.45                    156   
4        5     2.28                    175   
...    ...      ...                    ...   
1070  1071     2.26                    194   
1071  1072     2.36                    239   
1072  1073     2.26                    196   
1073  1074     2.41                    173   
1074  1075     2.39                    214   

      c-reactive_protein,_normal_and_high_sensitivity    wbc   rbc  \
0                                               1.830   6.91  4.97   
1                                               0.150   4.24  4.22   
2                                               0.137   6.38  4.31   
3                            

,id,gender,race,loc,cat,specific_diagnosis,ac_abn_od_cells,ac_abn_os_cells,vit_abn_od_cells,vit_abn_os_cells,vit_abn_od_haze,vit_abn_os_haze,calcium,lactate_dehydrogenase,"c-reactive_protein,_normal_and_high_sensitivity",wbc,rbc,hemoglobin,hematocrit,mcv,mch,mchc,rdw,platelet_count,neutrophil_%,lymphocytes_%,angiotensin_conv#enzyme,"lysozyme,_plasma",anti-ccp_ab,anti-ena_screen,antinuclear_antibody,complement_c3,complement_c4,dna_double-stranded_ab,rheumatoid_factor,hbc__ab,hbs__ag,hcv__ab
0,1,Male,Black/African American,anterior,Nonneoplastic Masquerade,idiopathic_anterior,0.0,0.0,0.0,0.5,0.0,0.0,2.27,184,1.830,6.91,4.97,14.6,41.4,83.3,29.4,35.3,13.9,194,64.4,30.8,NaN,3.0,<20,NEG,NEG,149.0,33.0,NEG,15,0,0,0
1,2,Female,Black/African American,intermediate,Idiopathic,pars planitis,2.0,2.0,2.0,2.0,0.5,1.0,2.32,209,0.150,4.24,4.22,11.0,36.0,85.3,26.1,30.6,14.8,260,40.1,49.1,27.3,NaN,<20,NEG,NEG,130.8,22.8,NEG,10,0,0,0
2,3,Female,White,panuveitis,Systemic,vkh,0.5,0.5,1.0,1.0,0.0,0.0,2.44,194,0.137,6.38,4.31,13.6,38.7,89.8,31.6,35.1,11.7,344,70.2,22.1,60.2,4.8,<20,NEG,NEG,135.9,18.2,NEG,10,0,0,0
3,4,Female,White,anterior,Idiopathic,idiopathic_anterior,0.0,0.0,0.0,0.0,0.0,0.0,2.45,156,3.500,5.24,4.82,14.6,43.6,90.5,30.3,33.5,12.4,220,56.6,34.2,38.9,NaN,<20,NEG,NEG,130.0,19.9,NEG,15,0,0,0
4,5,Female,unknown,anterior,Idiopathic,idiopathic_anterior,0.0,0.0,0.0,0.0,0.0,0.0,2.28,175,1.000,8.31,4.72,14.0,40.7,86.2,29.7,34.4,12.1,324,60.4,27.4,18.3,4.5,<20,NEG,NEG,130.4,19.0,73,10,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1070,1071,Male,Asian,panuveitis,Infectious,tuberculosis,0.0,0.0,0.0,0.0,0.0,0.0,2.26,194,15.400,7.22,5.49,12.7,40.6,74.0,23.1,31.3,15.4,292,67.3,17.7,1.0,9.3,<20,NEG,NEG,112.4,36.1,NEG,15,0,0,0
1071,1072,Male,unknown,intermediate,Idiopathic,pars planitis,0.0,1.0,0.0,2.0,0.0,2.0,2.36,239,0.800,7.65,4.56,12.6,35.9,78.7,27.6,35.1,13.5,219,54.3,38.6,53.0,3.6,<20,NEG,NEG,121.5,22.3,NEG,15,0,0,0
1072,1073,Female,unknown,posterior,Infectious,toxoplasmosis,0.0,0.0,0.0,1.0,0.0,1.0,2.26,196,0.600,3.83,4.54,14.5,42.8,94.3,31.9,33.9,14.9,239,52.6,35.5,39.9,2.9,<20,NEG,NEG,114.9,24.6,NEG,15,0,0,0
1073,1074,Female,Asian,posterior,WDS,wds,0.0,0.0,0.0,0.0,0.0,0.0,2.41,173,6.970,16.65,4.94,13.4,40.5,82.0,27.1,33.1,15.7,369,70.5,24.3,23.0,3.6,<20,NEG,NEG,131.0,26.0,NEG,15,0,0,0


## Features containing both numerical and categorical values
Certain columns don't follow the tidy data principle that only one datatyp should be existant in a column/feature.
This chapter deals with said columns and either splits them into a numeric and categorical feature or changes values to reach a uniform datatyp over a column.

### Anti-CCP Ab
Anti-CCP is a numeric column with mostly values set to '<20'. A value below or at 20 is viewed as a negative result. Above 20 the result is positive. This allows for a binarization of the column. We set every value below or at 20 to 0 (aka 'negative') and all values above 20 to 1 (aka postive). Some values are still missing.

In [50]:
# df = pipe.num_to_binary(df, 'anti-ccp_ab', 20)
# df['anti-ccp_ab'].value_counts(dropna=False)

### Anti-ENA Screen
Anti-ENA Screen consists of mostly 'NEG' (Negative) Values (1001 out of 1075), we assume that the other, numerical values can be regarded as positive. We encode these into 0 (Negative) and 1 (Positive) values. The singel occurance of 'see note | In-house test down.  Test re-ordered and sent to Referral L' gets dropped and replaced with `np.nan`.

In [51]:
# df['anti-ena_screen'].value_counts(dropna=False)

### Antinuclear Antibody

In [52]:
# df['antinuclear_antibody'].value_counts()

### DNA Double-Stranded Ab

In [53]:
# df['dna_double-stranded_ab'].value_counts(dropna=False)

The function `pipe.neg_col_to_cat` transforms a list of columns (in our case `['anti-ena_screen','antinuclear_antibody','dna_double-stranded_ab']`) to binary, categorical columns where 0 = 'Negative' and 1 = 'Postive

In [54]:
df = pipe.neg_col_to_cat(df, ['anti-ena_screen','antinuclear_antibody','dna_double-stranded_ab'])

### Myeloperoxidase Ab
This column has been dropped because of to many missing values

In [55]:
# df['myeloperoxidase_ab'].value_counts(dropna=False)

### Proteinase-3 Antibodies
This column has been dropped because of to many missing values

In [56]:
# df['proteinase-3_antibodies'].value_counts(dropna=False)

## Drop 'uom' and 'range' columns
Every lab test is accompanied by two columns. One specifies the unit of measurement (uom) for said test and the other defines the acceptable/normal range of the test (range).
Although these informations are important for the exploratory data analysis test and the preprocessing it is not advised to include these columns in the dataframe that serves as the input for a machine learning algorithmn. 

In [57]:
# df = pipe.drop_uom_and_range(df, verbose=True)

# Preprocessing Pipeline

CHECK IF EVERY COLUMN IS ACCOUNTED FOR:
- 'id'
- [x] 'gender' 
- [x] 'race'
- [x] 'loc'
- [x] 'ehr_diagnosis' (Dropped)
- [x] 'anti-dnase_b' (Dropped, too many missing values)
- [x] 'other_' (Dropped, too many missing values)
- [x] 'notes' (Dropped, too many missing values)
- [x] 'beta-2-microglobulin' (Dropped, too many missing values)
- [x] 'lupus_anticoagulant' (Dropped, too many missing values)
- [x] 'myeloperoxidase_ab' (Dropped, too many missing values)
- [x] 'proteinase-3_antibodies' (Dropped, too many missing values)
- 'cat'
- [x] 'specific_diagnosis'
- [x] 'ac_abn_od_cells'
- [x] 'ac_abn_os_cells'
- [x] 'vit_abn_od_cells'
- [x] 'vit_abn_os_cells'
- [x] 'vit_abn_od_haze'
- [x] 'vit_abn_os_haze'
- 'calcium'
- 'lactate_dehydrogenase'
- 'c-reactive_protein,_normal_and_high_sensitivity'
- 'wbc'
- 'rbc'
- 'hemoglobin'
- 'hematocrit'
- 'mcv'
- 'mch'
- 'mchc'
- 'rdw'
- 'platelet_count'
- 'neutrophil_%'
- 'lymphocytes_%'
- 'angiotensin_conv#enzyme'
- 'lysozyme,_plasma'
- 'anti-ccp_ab'
- 'anti-ena_screen'
- 'antinuclear_antibody'
- 'complement_c3'
- 'complement_c4'
- 'dna_double-stranded_ab'
- [x] 'hla-a*' (Dropped)
- [x] 'hla_a_1' (Dropped)
- [x] 'hla_a_2' (Dropped)
- [x] 'hla-b*' (Dropped)
- [x] 'hla_b_1' (Dropped)
- [x] 'hla_b_2' (Dropped)
- [x] 'hla-cw*' (Dropped)
- [x] 'hla_c_1' (Dropped)
- [x] 'hla_c_2' (Dropped)
- [x] 'hla-drb1*' (Dropped)
- [x] 'hla_drb1_1' (Dropped)
- [x] 'hla_drb1_2' (Dropped)
- [x] 'hla-dqb1*_/_dq*' (Dropped)
- [x] 'hla_dq_1' (Dropped)
- [x] 'hla_dq_2' (Dropped)
- [x] 'hla-drb_*' (Dropped)
- [x] 'hla_drb*_1' (Dropped)
- [x] 'hla_drb*_2' (Dropped)
- 'rheumatoid_factor'
- [x] 'hbc__ab'
- [x] 'hbs__ag'
- [x] 'hcv__ab'
- uom and range columns (Dropped, after used for transformation)

In [58]:
def preprocessing_pipe():
    # load dataset
    df = pd.read_excel("../data/uveitis_data.xlsx")
    
    df = (df.pipe(pipe.rename, path="../data/col_names&data_type-Copy1.xlsx") # rename columns
        .pipe(pd.DataFrame.applymap, lambda x: x.strip() if isinstance(x, str) else x) # strip leading or trailing whitespace

        # dropping columns
        .pipe(pipe.drop_nan_columns, nan_percentage=.5, verbose = False) # drop columns with above nan_percantage missing values
        .pipe(pd.DataFrame.drop, columns=['ehr_diagnosis'])
        .pipe(pipe.drop_via_filter, filter_str = 'hla', verbose=False)
        
        .pipe(gender_dtype) # change dtype from 'gender' to catgory
        .pipe(race_processing) # collapse 'race' feature
        .pipe(pipe.preprocessing_loc, approach='multi', verbose=False) # use approach ='binary' for binary classification
        .pipe(preprocessing_specific) # collapse 'specific_diagnosis'
        .pipe(preprocessing_inflammation) # # transform collumns that contain information about severeness of inlamation
        .pipe(preprocessing_hepatitis) # clean and binarize hepatitis-columns
        
        # drop 'uom' amd 'range' columns after use
        .pipe(pipe.drop_uom_and_range, verbose=False)
    )
    return df

df = preprocessing_pipe()


In [59]:
pd.set_option('max_columns', None)
df

,id,gender,race,loc,cat,specific_diagnosis,ac_abn_od_cells,ac_abn_os_cells,vit_abn_od_cells,vit_abn_os_cells,vit_abn_od_haze,vit_abn_os_haze,calcium,lactate_dehydrogenase,"c-reactive_protein,_normal_and_high_sensitivity",wbc,rbc,hemoglobin,hematocrit,mcv,mch,mchc,rdw,platelet_count,neutrophil_%,lymphocytes_%,angiotensin_conv#enzyme,"lysozyme,_plasma",anti-ccp_ab,anti-ena_screen,antinuclear_antibody,complement_c3,complement_c4,dna_double-stranded_ab,rheumatoid_factor,hbc__ab,hbs__ag,hcv__ab
0,1,Male,Black/African American,anterior,Nonneoplastic Masquerade,idiopathic_anterior,0.0,0.0,0.0,0.5,0.0,0.0,2.27,184,1.83,6.91,4.97,14.6,41.4,83.3,29.4,35.3,13.9,194,64.4,30.8,NaN,3,<20,NEG,NEG,149.0,33,NEG,<15,0,0,0
1,2,Female,Black/African American,intermediate,Idiopathic,pars planitis,2.0,2.0,2.0,2.0,0.5,1.0,2.32,209,<0.15,4.24,4.22,11,36,85.3,26.1,30.6,14.8,260,40.1,49.1,27.3,NaN,<20,NEG,NEG,130.8,22.8,NEG,<10,0,0,0
2,3,Female,White,panuveitis,Systemic,vkh,0.5,0.5,1.0,1.0,0.0,0.0,2.44,194,0.137,6.38,4.31,13.6,38.7,89.8,31.6,35.1,11.7,344,70.2,22.1,60.2,4.8,<20,NEG,NEG,135.9,18.2,NEG,<10,0,0,0
3,4,Female,White,anterior,Idiopathic,idiopathic_anterior,0.0,0.0,0.0,0.0,0.0,0.0,2.45,156,3.5,5.24,4.82,14.6,43.6,90.5,30.3,33.5,12.4,220,56.6,34.2,38.9,:,<20,NEG,NEG,130.0,19.9,NEG,<15,0,0,0
4,5,Female,unknown,anterior,Idiopathic,idiopathic_anterior,0.0,0.0,0.0,0.0,0.0,0.0,2.28,175,1,8.31,4.72,14,40.7,86.2,29.7,34.4,12.1,324,60.4,27.4,18.3,4.5,<20,NEG,NEG,130.4,19,73,<10,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1070,1071,Male,Asian,panuveitis,Infectious,tuberculosis,0.0,0.0,0.0,0.0,0.0,0.0,2.26,194,15.4,7.22,5.49,12.7,40.6,74,23.1,31.3,15.4,292,67.3,17.7,< 1.0,9.3,<20,NEG,NEG,112.4,36.1,NEG,<15,0,0,0
1071,1072,Male,unknown,intermediate,Idiopathic,pars planitis,0.0,1.0,0.0,2.0,0.0,2.0,2.36,239,0.8,7.65,4.56,12.6,35.9,78.7,27.6,35.1,13.5,219,54.3,38.6,53,3.6,<20,NEG,NEG,121.5,22.3,NEG,<15,0,0,0
1072,1073,Female,unknown,posterior,Infectious,toxoplasmosis,0.0,0.0,0.0,1.0,0.0,1.0,2.26,196,0.6,3.83,4.54,14.5,42.8,94.3,31.9,33.9,14.9,239,52.6,35.5,39.9,2.9,<20,NEG,NEG,114.9,24.6,NEG,<15,0,0,0
1073,1074,Female,Asian,posterior,WDS,wds,0.0,0.0,0.0,0.0,0.0,0.0,2.41,173,6.97,16.65,4.94,13.4,40.5,82,27.1,33.1,15.7,369,70.5,24.3,23,3.6,<20,NEG,NEG,131.0,26,NEG,<15,0,0,0


In [60]:
numeric_features = []
categorical_features = []
imputer = {'categorical':{'strategy':'constant', 'fill_value':'missing'}, 'numerical':{'strategy':'median'}}
# preprocessor = pipe.preprocessing(categorical_features, numeric_features, imputer)